In [13]:
# import pandas as pd
# import pulp as pl

# # Veriyi okuma
# df1 = pd.read_csv("set1_new.csv")
# df2 = pd.read_csv("set2.csv")

In [ ]:
import pandas as pd
import pulp as pl
from collections import defaultdict
import random

# Veri okuma
df1 = pd.read_csv("set1_new.csv")
df2 = pd.read_csv("set2.csv")

# Kullanıcının yaş ve cinsiyet bilgisi
age = 20
gender = "Kadın"

# Gerekli sınırlamaları alıyoruz
limits = df2[(df2["Cinsiyet"] == gender) & (df2["Yaş Grubu"].apply(lambda x: age in range(*map(int, x.split('-')))))].iloc[0]

# Kategori bilgisi
categories = {
    "main": range(1, 72),
    "soup": range(72, 100),
    "half_main": range(100, 150),
    "dessert_salad": range(150, 210),
}

price_conversion = {'a': 5, 'b': 4, 'c': 3, 'd': 2, 'e': 1}

# Önceden belirlenmiş yemeklerin ID'lerini bulma
mandatory_dishes = {
    'Etli Nohut': {'ID': 1, 'category': 'main'},
    'Etli Kuru Fasulye': {'ID': 2, 'category': 'main'},
    'Zeytinyağlı Barbunya': {'ID': 3, 'category': 'main'},
    'Börülce Salatası': {'ID': 151, 'category': 'dessert_salad'},
    'Mercimek Salatası': {'ID': 152, 'category': 'dessert_salad'},
    'Piyaz': {'ID': 153, 'category': 'dessert_salad'},
}

# Haftalık olarak kullanılan zorunlu yemekleri saklayan bir set
used_mandatory_dishes = set()

# Her gün için kullanılan yemekleri saklamak için bir sözlük oluşturuyoruz
used_dishes = set()


# 4 hafta için 5 günlük menüler oluşturma
for week in range(1, 5):
    print(f"{week}. Hafta Menüleri\n")
    
    mandatory_dish_days = set()
    
    for day in range(1, 6):
        prob = pl.LpProblem(f"Menu_Optimization_{week}_{day}", pl.LpMinimize)
        
        available_dishes = list(set(df1['ID']) - used_dishes)
        dish_vars = pl.LpVariable.dicts("Dish", available_dishes, 0, 1, pl.LpBinary)

        # Amaç fonksiyonu
        prob += pl.lpSum([price_conversion[df1.set_index("ID").loc[i]["Fiyat"]] * dish_vars[i] for i in available_dishes])

        # Besin değeri kısıtlamaları
        for nutrient in ["Enerji", "Karbonhidrat", "Protein", "Yağ", "Lif"]:
            lower_limit = float(limits[nutrient].split('-')[0])
            upper_limit = float(limits[nutrient].split('-')[1])
            
            prob += pl.LpConstraint(
                e=pl.lpSum([df1.set_index("ID").loc[i][nutrient] * dish_vars[i] for i in available_dishes]),
                sense=pl.LpConstraintGE,
                name=f"{nutrient}_lower_bound",
                rhs=lower_limit
            )
            
            prob += pl.LpConstraint(
                e=pl.lpSum([df1.set_index("ID").loc[i][nutrient] * dish_vars[i] for i in available_dishes]),
                sense=pl.LpConstraintLE,
                name=f"{nutrient}_upper_bound",
                rhs=upper_limit
            )

        # Kategori kısıtlamaları
        for cat, rng in categories.items():
            prob += pl.lpSum([dish_vars[i] for i in available_dishes if i in rng]) == 1

  
        for dish_name in mandatory_dish_names:
            dish_id = mandatory_dishes[dish_name]['ID']
            dish_category = mandatory_dishes[dish_name]['category']

            if dish_id not in used_dishes and day not in mandatory_dish_days:
                print(f"{week}. Hafta, {day}. Gün: Zorunlu yemek eklendi: {dish_name}")
                mandatory_dish_days.add(day)
                
                dish_vars[dish_id].varValue = 1
                used_dishes.add(dish_id)
                
                prob += pl.lpSum([dish_vars[i] for i in available_dishes if i in categories[dish_category]]) == 1

        prob.solve(pl.PULP_CBC_CMD(msg=False))
         
        print("Status:", pl.LpStatus[prob.status])

        # Çözümü yazdırma ve toplam besin değerlerini saklama
        print(f"{day}. Gün Menüsü:")
        total_nutrients = defaultdict(float)
        sorted_dishes = defaultdict(dict)
        
        for v in prob.variables():
            if v.varValue == 1:
                dish_id = int(v.name.split('_')[1])
                dish_info = df1.set_index("ID").loc[dish_id]
                
                used_dishes.add(dish_id)
                
                for cat, rng in categories.items():
                    if dish_id in rng:
                        sorted_dishes[cat] = {
                            "ID": dish_id,
                            "Yemek": dish_info['Yiyecek adı'],
                            "Fiyat": dish_info['Fiyat'],
                        }
                
                for nutrient in ["Enerji", "Karbonhidrat", "Protein", "Yağ", "Lif"]:
                    total_nutrients[nutrient] += dish_info[nutrient]
        
        for cat in ["main", "soup", "half_main", "dessert_salad"]:
            dish = sorted_dishes.get(cat)
            if dish:
                print(f"{cat.upper()}: ID: {dish['ID']}, Yemek: {dish['Yemek']}, Fiyat: {dish['Fiyat']}")
            else:
                print(f"{cat.upper()}: Bu kategoride yemek seçilememiştir.")
            # print(f"{cat.upper()}: ID: {dish['ID']}, Yemek: {dish['Yemek']}, Fiyat: {dish['Fiyat']}")
        
        print("Toplam Besin Değerleri:")
        for nutrient, value in total_nutrients.items():
            print(f"{nutrient}: {value}")

        print("---")
    print("\n")


In [84]:
import pandas as pd
import pulp as pl
from collections import defaultdict

mandatory_dish_names = ["Etli Nohut", "Etli Kuru Fasulye", "Zeytinyağlı Barbunya", "Börülce Salatası", "Mercimek Salatası", "Piyaz"]

def find_mandatory_dish_id(df, used_dishes):
    for dish_name in mandatory_dish_names:
        dish_row = df[df['Yiyecek adı'] == dish_name]
        if not dish_row.empty:
            dish_id = dish_row.iloc[0]['ID']
            if dish_id not in used_dishes:
                return dish_id, dish_name
    return None, None


df1 = pd.read_csv("set1_new.csv")
df2 = pd.read_csv("set2.csv")

age = 20
gender = "Kadın"

limits = df2[(df2["Cinsiyet"] == gender) & (df2["Yaş Grubu"].apply(lambda x: age in range(*map(int, x.split('-')))))].iloc[0]
mandatory_dish_names = ["Etli Nohut", "Etli Kuru Fasulye", "Zeytinyağlı Barbunya", "Börülce Salatası", "Mercimek Salatası", "Piyaz"]
mandatory_dish_ids = [df1[df1['Yiyecek adı'] == name].iloc[0]['ID'] for name in mandatory_dish_names if not df1[df1['Yiyecek adı'] == name].empty]

categories = {
    "main": range(1, 72),
    "soup": range(72, 100),
    "half_main": range(100, 150),
    "dessert_salad": range(150, 210),
}

price_conversion = {'a': 5, 'b': 4, 'c': 3, 'd': 2, 'e': 1}

used_dishes = set()
used_mandatory_dishes = set()


for week in range(1, 5):
    print(f"{week}. Hafta Menüleri\n")
    # mandatory_dish_days = set()

    mandatory_dish_day = random.choice(range(1, 6))
    mandatory_dish_id, mandatory_dish_name = None, None
    
    for day in range(1, 6):
        prob = pl.LpProblem(f"Menu_Optimization_{week}_{day}", pl.LpMinimize)
        available_dishes = list(set(df1['ID']) - used_dishes)
        dish_vars = pl.LpVariable.dicts("Dish", available_dishes, 0, 1, pl.LpBinary)

        prob += pl.lpSum([price_conversion[df1.set_index("ID").loc[i]["Fiyat"]] * dish_vars[i] for i in available_dishes])
        
        for nutrient in ["Enerji", "Karbonhidrat", "Protein", "Yağ", "Lif"]:
            lower_limit = float(limits[nutrient].split('-')[0])
            upper_limit = float(limits[nutrient].split('-')[1])
            
            prob += pl.LpConstraint(
                e=pl.lpSum([df1.set_index("ID").loc[i][nutrient] * dish_vars[i] for i in available_dishes]),
                sense=pl.LpConstraintGE,
                name=f"{nutrient}_lower_bound",
                rhs=lower_limit
            )
            
            prob += pl.LpConstraint(
                e=pl.lpSum([df1.set_index("ID").loc[i][nutrient] * dish_vars[i] for i in available_dishes]),
                sense=pl.LpConstraintLE,
                name=f"{nutrient}_upper_bound",
                rhs=upper_limit
            )

        for cat, rng in categories.items():
            prob += pl.lpSum([dish_vars[i] for i in available_dishes if i in rng]) == 1

        # Zorunlu yemeği ekleyelim
        if day == mandatory_dish_day:
            mandatory_dish_id, mandatory_dish_name = find_mandatory_dish_id(df1, used_mandatory_dishes)
            if mandatory_dish_id is not None:
                prob += dish_vars[mandatory_dish_id] == 1
                used_mandatory_dishes.add(mandatory_dish_id)  # bu yemeği ekledik
                print(f"Zorunlu yemek {mandatory_dish_id} ({mandatory_dish_name}) hafta {week}, gün {day} için eklendi.")


        
        prob.solve(pl.PULP_CBC_CMD(msg=False))
        print("Status:", pl.LpStatus[prob.status])

        total_nutrients = defaultdict(float)
        
        for v in prob.variables():
            if v.varValue == 1:
                dish_id = int(v.name.split('_')[1])
                dish_info = df1.set_index("ID").loc[dish_id]
                print(f"ID: {dish_id}, Yemek: {dish_info['Yiyecek adı']}, Fiyat: {dish_info['Fiyat']}")
                used_dishes.add(dish_id)
                
                for nutrient in ["Enerji", "Karbonhidrat", "Protein", "Yağ", "Lif"]:
                    total_nutrients[nutrient] += dish_info[nutrient]
        
        print("Toplam Besin Değerleri:")
        for nutrient, value in total_nutrients.items():
            print(f"{nutrient}: {value}")
        
        print("---")


1. Hafta Menüleri

Status: Optimal
ID: 138, Yemek: Havuç Soslu Makarna, Fiyat: e
ID: 172, Yemek: Yoğurt, Fiyat: e
ID: 71, Yemek: Kremalı Domates Çorbası, Fiyat: e
ID: 74, Yemek: Mercimek Çorbası, Fiyat: e
Toplam Besin Değerleri:
Enerji: 597.6
Karbonhidrat: 60.400000000000006
Protein: 24.7
Yağ: 27.599999999999998
Lif: 8.5
---
Status: Optimal
ID: 130, Yemek: Zeytinyağlı Enginar, Fiyat: e
ID: 196, Yemek: Balkabağı Tatlısı, Fiyat: e
ID: 27, Yemek: Tavuk Pane, Fiyat: d
ID: 94, Yemek: Pirinçli Kabak Çorba, Fiyat: e
Toplam Besin Değerleri:
Enerji: 594.7
Karbonhidrat: 61.4
Protein: 24.3
Yağ: 27.4
Lif: 12.0
---
Zorunlu yemek 55 (Etli Nohut) hafta 1, gün 3 için eklendi.
Status: Optimal
ID: 118, Yemek: Havuçlu Pilav, Fiyat: e
ID: 158, Yemek: Patates Salatası, Fiyat: e
ID: 55, Yemek: Etli Nohut, Fiyat: c
ID: 90, Yemek: Kelle Paça Çorbası, Fiyat: e
Toplam Besin Değerleri:
Enerji: 578.4
Karbonhidrat: 56.300000000000004
Protein: 25.200000000000003
Yağ: 27.5
Lif: 11.499999999999998
---
Status: Optimal